In [1]:
from UnSu_feature import Encoder, build_dataset_RR, batch_generator
## unsupervised feature ##
import sys
sys.path.append('..')
sys.path.append('../..')
from Common.Dataset_generator import DatasetsNMResidualBlockToOne
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
Loss = nn.MSELoss()

In [2]:
horizon = 62
epoch = 200
batch_size = 64

In [3]:
net = Encoder(horizon, 2, batch_size).to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-4)

D = DatasetsNMResidualBlockToOne('../../data/trajectories', horizon, False, False, subsample=1, filter=False)
# path:str, horizon:int, overlapping:bool, shuffle_dataset:bool, subsample: sampling gap, filter: smoothness

Embedding dimension:  217
<Common.Dataset_generator.DatasetsNMResidualBlockToOne object at 0x7f71302fc5c0> : no mean-var initialization!


In [ ]:
Ytrain, Xtrain_noExpand = build_dataset_RR(D, horizon)
input_feature = Xtrain_noExpand.shape[1]
for _ in range(epoch):
    Ls = 0
    for x in batch_generator(Xtrain_noExpand, batch_size):
        pred = net(x)
        optimizer.zero_grad()
        L = Loss(pred, x)
        Ls += L.item()
        L.backward()
        optimizer.step()

    print("Loss: ", Ls)

Loss:  1874.755908750274
Loss:  1711.6222645636444
Loss:  1626.3152010039485
Loss:  1550.96534699318
Loss:  1487.4335896265984
Loss:  1449.84918447565
Loss:  1427.882852458686
Loss:  1401.2963457631413
Loss:  1372.8136721751252
Loss:  1369.7621649244902
Loss:  1349.1965055810979
Loss:  1345.2335401303244
Loss:  1332.3371442557436
Loss:  1325.7276671147192
Loss:  1325.1353032511324
Loss:  1313.8734246611566
Loss:  1312.0473103991717
Loss:  1305.9398168376392
Loss:  1295.7810545334687
Loss:  1300.2197080704136
Loss:  1281.6798733699761
Loss:  1274.0591681799415
Loss:  1279.237739582768
Loss:  1287.6513223593684
Loss:  1284.1931077196423
Loss:  1273.5641972322228
Loss:  1280.790027082261
Loss:  1274.3904712333715


In [38]:
for idx, (name, param) in enumerate(net.named_parameters()):
    if idx==0:
        np.savetxt('W1.txt', param.detach().cpu().numpy())
    if idx==1:
        np.savetxt('b1.txt', param.detach().cpu().numpy())
    if idx==2:
        np.savetxt('W2.txt', param.detach().cpu().numpy())
    if idx==3:
        np.savetxt('b2.txt', param.detach().cpu().numpy())